<a href="https://colab.research.google.com/github/mrhamza000/Adminlte/blob/main/RealTime_Forcasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

section 1: **Import labraries**

In [3]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime, timedelta
import pytz


In [4]:
API_KEY = 'b3d22e1d3f281248f3f5a4cf88f4381c'
BASE_URL = 'https://api.openweathermap.org/data/2.5/'


**1.Fetch Current Weather Data**

In [5]:
def get_current_weather(city):
  url = f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response = requests.get(url)
  data = response.json()
  return {
      'city': data['name'],
      'current_temp': round(data['main']['temp']),
      'feels_like': round(data['main']['feels_like']),
      'min_temp': round(data['main']['temp_min']),
      'max_temp': round(data['main']['temp_max']),
      'humidity': round(data['main']['humidity']),
      'description': data['weather'][0]['description'],
      'country': data['sys']['country'],
      'wind_gust_dir': data['wind']['deg'],
      'pressure': data['main']['pressure'],
      'wind_gust_speed': data['wind']['speed'],


  }



**2. Read Data**

In [6]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df = df.dropna()
  df = df.drop_duplicates()
  return df


**3. Prepare data for training**

In [7]:
def prepare_data(data):
  le = LabelEncoder()
  data['WindGustDir'] = le.fit_transform(data['WindGustDir'])
  data['RainTomorrow'] = le.fit_transform(data['RainTomorrow'])


  #define the feature Variable and Target variable
  X = data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure','Temp']]
  y = data['RainTomorrow']
  return X, y, le


**4. Train Rain Prediction Model**

In [8]:
def train_rain_model(X,y):
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(X_train, y_train)
  y_pred = model.predict(X_test)
  print("Mean Squared Error for Rain Model")
  print(mean_squared_error(y_test, y_pred))
  return model

**5. Prepare Regression Data**

In [9]:
def prepare_regression_data(data, feature):
  X, y = [], []
  for i in range(len(data) - 1):
    X.append(data[feature].iloc[i])
    y.append(data[feature].iloc[i+1])

  X = np.array(X).reshape(-1,1)
  y = np.array(y)
  return X, y


**6. Train Regression Model**

In [10]:
def train_regression_model(X,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(X,y)
  return model

**7. Predict Future**

In [11]:
def predict_future(model, current_value):
  predictions = [current_value]

  for i in range(5):
    next_value = model.predict(np.array([[predictions[-1]]]))

    predictions.append(next_value[0])
  return predictions[1:]

**8. Weather Analysis Function**

In [41]:
from time import timezone
def weather_view():
  city = input("Enter the city name: ")
  current_weather = get_current_weather(city)

  historical_data = read_historical_data('/content/weather.csv')

  #prepare and train the rain prediction model
  X, y, le = prepare_data(historical_data)
  rain_model = train_rain_model(X, y)

  #map wind direction to campass points
  wind_deg = current_weather['wind_gust_dir'] % 360
  compass_points = [
      ("N",0, 11.25), ("NNE",11.25,33.75), ("NE",33.75,56.25),
      ("ENE",56.25,78.75), ("E",78.75,101.25), ("ESE",101.25,123.75),
      ("SE",123.75,146.25), ("SSE",146.25,168.75), ("S",168.75,191.25),
      ("SSW",191.25,213.75), ("SW",213.75,236.25), ("WSW",236.25,258.75),
      ("W",258.75,281.25), ("WNW",281.25,303.75), ("NW",303.75,326.25),
      ("NNW",326.25,348.75)
  ]
  compass_direction = next(point for point, start, end in compass_points if start <= wind_deg < end)
  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['min_temp'],
      'MaxTemp': current_weather['max_temp'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['wind_gust_speed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp']
  }
  current_df = pd.DataFrame([current_data])

  #Rain prediction
  rain_prediction = rain_model.predict(current_df)[0]

  #prepare regression model for temprature and humidity
  X_temp, y_temp = prepare_regression_data(historical_data, 'Temp')
  X_hum, y_hum = prepare_regression_data(historical_data, 'Humidity')

  temp_model = train_regression_model(X_temp, y_temp)
  hum_model = train_regression_model(X_hum, y_hum)

  #Predict future temprature
  future_temp = predict_future(temp_model, current_weather['min_temp'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  #prepare time for future prediction
  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour = now + timedelta(hours=1)
  next_hour = next_hour.replace(minute=0, second=0, microsecond=0)

  future_times = [(next_hour + timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  #Display the result

  print(f"Current Weather in {city}, {current_weather['country']}")
  print(f"Current Temperature: {current_weather['current_temp']}°C")
  print(f"Feels Like: {current_weather['feels_like']}")
  print(f"Min Temperature: {current_weather['min_temp']}°C")
  print(f"Max Temperature: {current_weather['max_temp']}°C")
  print(f"Humidity: {current_weather['humidity']}%")
  print(f"Description: {current_weather['description']}")
  print(f"Rain Prediction: {'Yes' if rain_prediction else 'No'}")


  print("\nFuture Temperature Predictions:")
  for time, temp in zip(future_times, future_temp):
    print(f"{time}: {round(temp, 1)}°C")

  print("\nFuture Humidity Predictions:")

  for time, humidity in zip(future_times, future_humidity):
    print(f"{time}: {round(humidity, 1)}%")



weather_view()







Enter the city name: manchester
Mean Squared Error for Rain Model
0.1506849315068493
Current Weather in manchester, GB
Current Temperature: 13°C
Feels Like: 12
Min Temperature: 12°C
Max Temperature: 15°C
Humidity: 57%
Description: broken clouds
Rain Prediction: No

Future Temperature Predictions:
17:00: 12.8°C
18:00: 12.4°C
19:00: 14.4°C
20:00: 13.4°C
21:00: 13.6°C

Future Humidity Predictions:
17:00: 40.7%
18:00: 52.9%
19:00: 51.0%
20:00: 46.6%
21:00: 47.2%
